In [65]:
import pandas as pd
import numpy as np
import torch
import sklearn
import torch.nn.functional as F

In [66]:
df = pd.read_csv("AWE_1000_score.csv")

In [67]:
df.head(2)

,Unnamed: 0,id,nationality,sex,title,content,score
0,1,A100000_v01.txt_2.txt,중국,남자,우리 나라의 계절과 날씨,제 나라의 계절은 아주 좋아요. 네 계절이 있습니다. 봄에는 날씨가 좋아요. 꽃이 ...,80
1,2,A100000_v02.txt_1.txt,중국,여자,주말 이야기,어제 명동에 갔습니다. 친구하고 같이 갔습니다. 가게가 많습니다. 옷가게가 있었습니...,70


In [68]:
df["sentence"] = df["content"].apply(lambda x: len(x.split(".")))
#df["content"].apply(lambda x: len(x.split("."))).value_counts()

In [69]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = df["content"]
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)

In [70]:
len(vectorizer.get_feature_names_out())

8093

In [71]:
X

<1000x8093 sparse matrix of type '<class 'numpy.float64'>'
	with 34048 stored elements in Compressed Sparse Row format>

In [79]:
df["content_vector"]= X.todense().tolist()

In [80]:
df.head(2)

,Unnamed: 0,id,nationality,sex,title,content,score,sentence,content_vector
0,1,A100000_v01.txt_2.txt,중국,남자,우리 나라의 계절과 날씨,제 나라의 계절은 아주 좋아요. 네 계절이 있습니다. 봄에는 날씨가 좋아요. 꽃이 ...,80,18,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,2,A100000_v02.txt_1.txt,중국,여자,주말 이야기,어제 명동에 갔습니다. 친구하고 같이 갔습니다. 가게가 많습니다. 옷가게가 있었습니...,70,10,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [89]:
class MyLinearModel(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.linear = torch.nn.Linear(in_features=8093, out_features= 1)

  def forward(self, x):
    return self.linear(x)

In [90]:
myLin = MyLinearModel()

In [91]:
optimizer = torch.optim.SGD(myLin.parameters(), lr=0.01)

In [100]:
nb_epochs = 10
for epoch in range(nb_epochs + 1):
  total_loss = 0
  for idx in range(len(df)):
    x_train = np.array([df["content_vector"][idx]])
    y_train = np.array([df["score"][idx]])
    x_train = torch.from_numpy(x_train).float()
    y_train = torch.from_numpy(y_train).int()
    hypothesis = myLin(x_train)
    # cost 계산
    cost = F.mse_loss(hypothesis.to(torch.float32), y_train.to(torch.float32))
    total_loss = total_loss + cost.item()
    # cost로 H(x) 개선 
    optimizer.zero_grad() 
    cost.backward() 
    optimizer.step()
  print(epoch, total_loss / len(df))

<ipython-input-100-0cdedf403e2c>:11: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  cost = F.mse_loss(hypothesis.to(torch.float32), y_train.to(torch.float32))


0 175.76021923838323
1 167.82148048308608
2 160.50617327039515
3 153.741971000402
4 147.4671060589794
5 141.6285951287909
6 136.18078219308686
7 131.08420806209884
8 126.3045435849356
9 121.81189155275398
10 117.58005926325976
